In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Add, Activation, GlobalAveragePooling2D, Multiply
from tensorflow.keras.models import Model
from kerastuner import HyperModel, RandomSearch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def residual_block(x, filters):
    shortcut = x
    x = Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    return x

def attention_block(x, filters):
    avg_pool = GlobalAveragePooling2D()(x)
    dense1 = Dense(filters // 8, activation='relu')(avg_pool)
    dense2 = Dense(filters, activation='sigmoid')(dense1)
    x = Multiply()([x, dense2])
    return x

class MyHyperModel(HyperModel):
    def build(self, hp):
        input_shape = (4, 1, 1)
        inputs = Input(shape=input_shape)
        x = Conv2D(
            filters=hp.Int('initial_filters', min_value=16, max_value=64, step=16),
            kernel_size=(3, 3),
            padding='same',
            activation='relu'
        )(inputs)

        for i in range(hp.Int('num_blocks', 1, 3)):
            filters = hp.Int(f'filters_{i}', min_value=32, max_value=128, step=32)
            x = residual_block(x, filters)
            x = attention_block(x, filters)

        x = GlobalAveragePooling2D()(x)
        x = Dense(units=hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu')(x)
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs, outputs)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
            ),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        return model

data = pd.read_excel('path_to_your_file.xlsx')

X = data[['RLM Short Run Emphasis', 'RLM Run Length Nonuniformity', 'Gray Level Entropy Matrix Entropy', 'DFT Magnitude Spectrum Mean']].values
y = data['Status'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.reshape(-1, 4, 1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from kerastuner.tuners import RandomSearch
hypermodel = MyHyperModel()

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=12,
    executions_per_trial=2,
    directory='hyperparam_tuning',
    project_name='msa_rcnn'
)

tuner.search(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
best_model = tuner.get_best_models(num_models=1)[0]
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

print(f'Test accuracy: {test_accuracy:.4f}')
